## First is to scrape the table from Wikipedia page

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
!pip install bs4
from bs4 import BeautifulSoup

In [3]:
conda install lxml

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2020.4.5.1         |           py36_0         155 KB
    libxslt-1.1.33             |       h7d1a2b0_0         426 KB
    lxml-4.5.0                 |   py36hefd8a0e_0         1.4 MB
    openssl-1.1.1f             |       h7b6447c_0         2.5 MB
    ------------------------------------------------------------
                                           Total:         4.6 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               pkgs/main/linux-64::lxml-4.5.0-py36hefd8a0e_0

The following packages will be SUPERSEDED by a higher-priority channel:



In [32]:
# scraping the table and turning it into pandas df
result = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]
df = pd.DataFrame(df)
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [33]:
# renaming the columns
df.columns = ['PostalCode', 'Borough', 'Neighborhood'] 

# dropping 'Not assigned' values
df = df[df.Borough != 'Not assigned']

In [34]:
# function to replace '/' with comma
def replace_with_comma(text):
    text = text.replace(' /',',')
    return text

In [35]:
# avoiding SettingWithCopyWarning 
df = df.copy()

# replacing '/' with comma
df['Neighborhood'] = df['Neighborhood'].apply(replace_with_comma)

In [8]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [36]:
# checking if we have NaN values left in Neighborhood column
df['Neighborhood'].isnull().value_counts()

False    103
Name: Neighborhood, dtype: int64

In [37]:
df.shape

(103, 3)

### we need to get the latitude and the longitude coordinates of each neighborhood.

In [38]:
#Get data lat/long data from csv
coordinates = pd.read_csv("http://cocl.us/Geospatial_data")

#rename columns and set the index to be Postcode
coordinates.columns = ["PostalCode", "Latitude", "Longitude"]
if(coordinates.index.name != 'PostalCode'):
    coordinates = coordinates.set_index('PostalCode')
    
coordinates.head()

,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [39]:
# merge dataframes
df = df.merge(coordinates, how = 'left', on='PostalCode')
df.shape

(103, 5)

In [42]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
